In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer 
import stanfordnlp
import pickle
import gensim


# stanfordnlp.download('en')
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

In [2]:
tweets = pd.read_csv("uk_ru_2023_en_text_random_labeled_dataset_Labelled.csv",encoding="utf-8")

C:\Users\99zhe\AppData\Local\Temp\ipykernel_46416\2020894419.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  tweets = pd.read_csv("uk_ru_2023_en_text_random_labeled_dataset_Labelled.csv",encoding="utf-8")


In [3]:
tweets.head()

,Unnamed: 0,userid,tweetid,text,hashtags,token,processed_text,Topic Label,Topic Weight,Sentiment
0,0,5.467372e+07,1.627620e+18,repdavid i support 100,"SlavaUkraïni,JoeBiden","['repdavid', 'support']",repdavid support,International Support and Public Opinion,0.389973,Positive
1,1,2.282984e+09,1.611400e+18,province editorial inequality hounds s populat...,"Canada,disabled,Vancouver,homeless","['province', 'editorial', 'inequality', 'hound...",province editorial inequality hound population...,Strategic Military Aspects,0.464470,Irrelevant
2,2,2.830324e+08,1.636650e+18,they went to war when russia invaded ukraine t...,ArmUkraineNow,"['went', 'war', 'russia', 'invaded', 'ukraine'...",went war russia invaded ukraine believed freed...,Strategic Military Aspects,0.613352,Positive
3,3,1.257760e+18,1.626920e+18,on mintwave radio mint2dry,"nowplaying,alexa,streema,Scotland,echo,global,...","['mintwave', 'radio', 'mintdry']",mintwave radio mintdry,Military Actions and Operations,0.279435,Irrelevant
4,4,9.241172e+08,1.640020e+18,military historian douglas macgregor why will ...,"NATO,Russia,Kiev","['military', 'historian', 'douglas', 'macgrego...",military historian douglas macgregor crumble d...,Broad Overview of the Conflict,0.434376,Negative


In [4]:
def relevance(x):
    if x in ["Positive","Negative","Neutral"]:
        return "Relevant"
    else:
        return "Irrelevant"

In [5]:
tweets["Relevance"] = tweets["Sentiment"].apply(relevance)

In [6]:
tweets = tweets.iloc[: , 1:]
tweets.head()

,userid,tweetid,text,hashtags,token,processed_text,Topic Label,Topic Weight,Sentiment,Relevance
0,5.467372e+07,1.627620e+18,repdavid i support 100,"SlavaUkraïni,JoeBiden","['repdavid', 'support']",repdavid support,International Support and Public Opinion,0.389973,Positive,Relevant
1,2.282984e+09,1.611400e+18,province editorial inequality hounds s populat...,"Canada,disabled,Vancouver,homeless","['province', 'editorial', 'inequality', 'hound...",province editorial inequality hound population...,Strategic Military Aspects,0.464470,Irrelevant,Irrelevant
2,2.830324e+08,1.636650e+18,they went to war when russia invaded ukraine t...,ArmUkraineNow,"['went', 'war', 'russia', 'invaded', 'ukraine'...",went war russia invaded ukraine believed freed...,Strategic Military Aspects,0.613352,Positive,Relevant
3,1.257760e+18,1.626920e+18,on mintwave radio mint2dry,"nowplaying,alexa,streema,Scotland,echo,global,...","['mintwave', 'radio', 'mintdry']",mintwave radio mintdry,Military Actions and Operations,0.279435,Irrelevant,Irrelevant
4,9.241172e+08,1.640020e+18,military historian douglas macgregor why will ...,"NATO,Russia,Kiev","['military', 'historian', 'douglas', 'macgrego...",military historian douglas macgregor crumble d...,Broad Overview of the Conflict,0.434376,Negative,Relevant


In [7]:
lemmatizer = WordNetLemmatizer()

for i in range(0,tweets.shape[0]):
    sent = tweets.iloc[[i]]["token"]
    sent = [lemmatizer.lemmatize(w) for w in sent]
    tweets.iloc[[i]]["token"] = sent

C:\Users\99zhe\AppData\Local\Temp\ipykernel_46416\2160210669.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets.iloc[[i]]["token"] = sent


In [8]:
labelled_tweets = tweets[~tweets["Sentiment"].isnull()]
labelled_tweets.count()

userid            5000
tweetid           5000
text              5000
hashtags          4975
token             5000
processed_text    4999
Topic Label       5000
Topic Weight      5000
Sentiment         5000
Relevance         5000
dtype: int64

In [9]:
labelled_tweets.iloc[0:3000]

,userid,tweetid,text,hashtags,token,processed_text,Topic Label,Topic Weight,Sentiment,Relevance
0,5.467372e+07,1.627620e+18,repdavid i support 100,"SlavaUkraïni,JoeBiden","['repdavid', 'support']",repdavid support,International Support and Public Opinion,0.389973,Positive,Relevant
1,2.282984e+09,1.611400e+18,province editorial inequality hounds s populat...,"Canada,disabled,Vancouver,homeless","['province', 'editorial', 'inequality', 'hound...",province editorial inequality hound population...,Strategic Military Aspects,0.464470,Irrelevant,Irrelevant
2,2.830324e+08,1.636650e+18,they went to war when russia invaded ukraine t...,ArmUkraineNow,"['went', 'war', 'russia', 'invaded', 'ukraine'...",went war russia invaded ukraine believed freed...,Strategic Military Aspects,0.613352,Positive,Relevant
3,1.257760e+18,1.626920e+18,on mintwave radio mint2dry,"nowplaying,alexa,streema,Scotland,echo,global,...","['mintwave', 'radio', 'mintdry']",mintwave radio mintdry,Military Actions and Operations,0.279435,Irrelevant,Irrelevant
4,9.241172e+08,1.640020e+18,military historian douglas macgregor why will ...,"NATO,Russia,Kiev","['military', 'historian', 'douglas', 'macgrego...",military historian douglas macgregor crumble d...,Broad Overview of the Conflict,0.434376,Negative,Relevant
...,...,...,...,...,...,...,...,...,...,...
2995,1.226100e+18,1.633460e+18,will be next we told you this wasnt about just...,"Georgia,Moldova,Ukraine,Putin,Russia,RussiaIsa...","['told', 'wasnt', 'clear', 'want', 'restore', ...",told wasnt clear want restore root oppression,Military Actions and Operations,0.472631,Negative,Relevant
2996,1.496810e+18,1.636100e+18,a few good russians,Ukraine,"['good', 'russian']",good russian,Military Actions and Operations,0.326523,Positive,Relevant
2997,2.611555e+07,1.637140e+18,👏🏼👏🏼👏🏼we cant get enough military equipmentwea...,"PutinIsaWarCriminal,FCKPUTIN,RussiaIsATerroris...","['military', 'equipmentweapons', 'guy', 'fast'...",military equipmentweapons guy fast slava ukrai...,Strategic Military Aspects,0.471844,Positive,Relevant
2998,1.274051e+09,1.634670e+18,a russian mtlb carrying 100mm ammunition for t...,"Ukraine,Donetsk","['russian', 'mtlb', 'carrying', 'mm', 'ammunit...",russian mtlb carrying mm ammunition mt rapira ...,Military Actions and Operations,0.719004,Neutral,Relevant


In [10]:
train_tweets = labelled_tweets.iloc[0:3000]
test_tweets = labelled_tweets.iloc[3000:4000]
validate_tweets = labelled_tweets.iloc[4000:]

# Identifying Relevant and Irrelevant

With Topic and Hashtag

In [13]:
# Train Data with Topic and Hashtags

train_labels = train_tweets["Relevance"].tolist()
train_hashtags = train_tweets["hashtags"].tolist()
train_hashtags = [str(hashtags).split(",") for hashtags in train_hashtags] 
train_topics = train_tweets["Topic Label"].tolist()
train_corpus = list(train_tweets["token"])
train_corpus = [eval(i) for i in train_corpus] 

train_dictionary = gensim.corpora.Dictionary(train_corpus)

labeled_training_data = []
for (l, s, t, h) in zip(train_labels, train_corpus, train_topics, train_hashtags):
    vector = train_dictionary.doc2bow(s)
    sent_as_dict = {id:1 for (id, tf) in vector}
    sent_as_dict[t] = 1
    for tag in h:
        sent_as_dict[tag] = 1
    labeled_training_data.append((sent_as_dict, l))
    
classifierRelevance1NB = nltk.NaiveBayesClassifier.train(labeled_training_data)
classifierRelevance1M = nltk.MaxentClassifier.train(labeled_training_data)

  ==> Training (100 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -0.69315        0.741
             2          -0.30649        0.852
             3          -0.23281        0.937
             4          -0.19160        0.967
             5          -0.16453        0.975
             6          -0.14504        0.981
             7          -0.13018        0.985
             8          -0.11840        0.988
             9          -0.10877        0.989
            10          -0.10073        0.990
            11          -0.09391        0.991
            12          -0.08803        0.992
            13          -0.08291        0.993
            14          -0.07840        0.994
            15          -0.07439        0.995
            16          -0.07081        0.995
            17          -0.06759        0.995
            18          -0.06468        0.995
            19          -0.06202        0.995
 

In [14]:
# Test Data with Topic and Hashtags

test_labels = test_tweets["Relevance"].tolist()
test_hashtags = test_tweets["hashtags"].tolist()
test_hashtags = [str(hashtags).split(",") for hashtags in test_hashtags] 
test_topics = test_tweets["Topic Label"].tolist()
test_corpus = list(test_tweets["token"])
test_corpus = [eval(i) for i in test_corpus] 

test_dictionary = gensim.corpora.Dictionary(test_corpus)

labeled_test_data = []
for (l, s, t, h) in zip(test_labels, test_corpus, test_topics, test_hashtags):
    vector = test_dictionary.doc2bow(s)
    sent_as_dict = {id:1 for (id, tf) in vector}
    sent_as_dict["t"] = 1
    for tag in h:
        sent_as_dict[tag] = 1
    labeled_test_data.append((sent_as_dict, l))
    
print("NB Accuracy on Relevance test data: ", nltk.classify.accuracy(classifierRelevance1NB, labeled_test_data))
versionRelevance1NBacc = nltk.classify.accuracy(classifierRelevance1NB, labeled_test_data)
print("M Accuracy on Relevance test data: ", nltk.classify.accuracy(classifierRelevance1M, labeled_test_data))
versionRelevance1Macc = nltk.classify.accuracy(classifierRelevance1M, labeled_test_data)

NB Accuracy on Relevance test data:  0.647
M Accuracy on Relevance test data:  0.769


With Hashtags only

In [15]:
# Train Data with Hashtags

train_labels = train_tweets["Relevance"].tolist()
train_hashtags = train_tweets["hashtags"].tolist()
train_hashtags = [str(hashtags).split(",") for hashtags in train_hashtags] 
train_corpus = list(train_tweets["token"])
train_corpus = [eval(i) for i in train_corpus] 

train_dictionary = gensim.corpora.Dictionary(train_corpus)

labeled_training_data = []
for (l, s, h) in zip(train_labels, train_corpus, train_hashtags):
    vector = train_dictionary.doc2bow(s)
    sent_as_dict = {id:1 for (id, tf) in vector}
    for tag in h:
        sent_as_dict[tag] = 1
    labeled_training_data.append((sent_as_dict, l))
    
classifierRelevance2NB = nltk.NaiveBayesClassifier.train(labeled_training_data)
classifierRelevance2M = nltk.MaxentClassifier.train(labeled_training_data)

  ==> Training (100 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -0.69315        0.741
             2          -0.29599        0.864
             3          -0.22338        0.942
             4          -0.18341        0.968
             5          -0.15728        0.978
             6          -0.13852        0.983
             7          -0.12423        0.986
             8          -0.11290        0.988
             9          -0.10366        0.990
            10          -0.09595        0.991
            11          -0.08941        0.991
            12          -0.08378        0.994
            13          -0.07888        0.994
            14          -0.07457        0.995
            15          -0.07075        0.995
            16          -0.06733        0.995
            17          -0.06425        0.995
            18          -0.06147        0.995
            19          -0.05895        0.996
 

In [16]:
# Test Data with Hashtags

test_labels = test_tweets["Relevance"].tolist()
test_hashtags = test_tweets["hashtags"].tolist()
test_hashtags = [str(hashtags).split(",") for hashtags in test_hashtags] 
test_corpus = list(test_tweets["token"])
test_corpus = [eval(i) for i in test_corpus] 

test_dictionary = gensim.corpora.Dictionary(test_corpus)

labeled_test_data = []
for (l, s, h) in zip(test_labels, test_corpus, test_hashtags):
    vector = test_dictionary.doc2bow(s)
    sent_as_dict = {id:1 for (id, tf) in vector}
    for tag in h:
        sent_as_dict[tag] = 1
    labeled_test_data.append((sent_as_dict, l))
    
print("NB Relevance Accuracy on test data: ", nltk.classify.accuracy(classifierRelevance2NB, labeled_test_data))
versionRelevance2NBacc = nltk.classify.accuracy(classifierRelevance2NB, labeled_test_data)
print("M Relevance Accuracy on test data: ", nltk.classify.accuracy(classifierRelevance2M, labeled_test_data))
versionRelevance2Macc = nltk.classify.accuracy(classifierRelevance2M, labeled_test_data)

NB Relevance Accuracy on test data:  0.706
M Relevance Accuracy on test data:  0.781


With Topic only

In [17]:
# Train Data with Topic

train_labels = train_tweets["Relevance"].tolist()
train_topics = train_tweets["Topic Label"].tolist()
train_corpus = list(train_tweets["token"])
train_corpus = [eval(i) for i in train_corpus] 

train_dictionary = gensim.corpora.Dictionary(train_corpus)

labeled_training_data = []
for (l, s, t) in zip(train_labels, train_corpus, train_topics):
    vector = train_dictionary.doc2bow(s)
    sent_as_dict = {id:1 for (id, tf) in vector}
    sent_as_dict[t] = 1
    labeled_training_data.append((sent_as_dict, l))
    
classifierRelevance3NB = nltk.NaiveBayesClassifier.train(labeled_training_data)
classifierRelevance3M = nltk.MaxentClassifier.train(labeled_training_data)

  ==> Training (100 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -0.69315        0.741
             2          -0.33183        0.822
             3          -0.26392        0.918
             4          -0.22381        0.949
             5          -0.19653        0.961
             6          -0.17637        0.971
             7          -0.16067        0.977
             8          -0.14799        0.981
             9          -0.13747        0.984
            10          -0.12856        0.985
            11          -0.12091        0.987
            12          -0.11424        0.988
            13          -0.10836        0.989
            14          -0.10314        0.989
            15          -0.09847        0.990
            16          -0.09426        0.990
            17          -0.09044        0.990
            18          -0.08696        0.990
            19          -0.08377        0.991
 

In [18]:
# Test Data with Topic

test_labels = test_tweets["Relevance"].tolist()
test_topics = test_tweets["Topic Label"].tolist()
test_corpus = list(test_tweets["token"])
test_corpus = [eval(i) for i in test_corpus] 

test_dictionary = gensim.corpora.Dictionary(test_corpus)

labeled_test_data = []
for (l, s, t) in zip(test_labels, test_corpus, test_topics):
    vector = test_dictionary.doc2bow(s)
    sent_as_dict = {id:1 for (id, tf) in vector}
    sent_as_dict["t"] = 1
    labeled_test_data.append((sent_as_dict, l))
    
print("NB Relevance Accuracy on test data: ", nltk.classify.accuracy(classifierRelevance3NB, labeled_test_data))
versionRelevance3NBacc = nltk.classify.accuracy(classifierRelevance3NB, labeled_test_data)
print("M Relevance Accuracy on test data: ", nltk.classify.accuracy(classifierRelevance3M, labeled_test_data))
versionRelevance3Macc = nltk.classify.accuracy(classifierRelevance3M, labeled_test_data)

NB Relevance Accuracy on test data:  0.543
M Relevance Accuracy on test data:  0.546


Without Both

In [19]:
# Train Data with Topic

train_labels = train_tweets["Relevance"].tolist()
train_corpus = list(train_tweets["token"])
train_corpus = [eval(i) for i in train_corpus] 

train_dictionary = gensim.corpora.Dictionary(train_corpus)

labeled_training_data = []
for (l, s) in zip(train_labels, train_corpus):
    vector = train_dictionary.doc2bow(s)
    sent_as_dict = {id:1 for (id, tf) in vector}
    labeled_training_data.append((sent_as_dict, l))
    
classifierRelevance4NB = nltk.NaiveBayesClassifier.train(labeled_training_data)
classifierRelevance4M = nltk.MaxentClassifier.train(labeled_training_data)

  ==> Training (100 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -0.69315        0.741
             2          -0.31988        0.846
             3          -0.25227        0.929
             4          -0.21327        0.953
             5          -0.18695        0.966
             6          -0.16757        0.976
             7          -0.15250        0.979
             8          -0.14035        0.984
             9          -0.13028        0.986
            10          -0.12177        0.987
            11          -0.11447        0.988
            12          -0.10811        0.989
            13          -0.10252        0.990
            14          -0.09756        0.990
            15          -0.09313        0.991
            16          -0.08913        0.992
            17          -0.08551        0.992
            18          -0.08222        0.992
            19          -0.07920        0.993
 

In [20]:
# Test Data with Topic

test_labels = test_tweets["Relevance"].tolist()
test_corpus = list(test_tweets["token"])
test_corpus = [eval(i) for i in test_corpus] 

test_dictionary = gensim.corpora.Dictionary(test_corpus)

labeled_test_data = []
for (l, s) in zip(test_labels, test_corpus):
    vector = test_dictionary.doc2bow(s)
    sent_as_dict = {id:1 for (id, tf) in vector}
    labeled_test_data.append((sent_as_dict, l))
    
print("NB Relevance Accuracy on test data: ", nltk.classify.accuracy(classifierRelevance4NB, labeled_test_data))
versionRelevance4NBacc = nltk.classify.accuracy(classifierRelevance4NB, labeled_test_data)
print("M Relevance Accuracy on test data: ", nltk.classify.accuracy(classifierRelevance4M, labeled_test_data))
versionRelevance4Macc = nltk.classify.accuracy(classifierRelevance4M, labeled_test_data)

NB Relevance Accuracy on test data:  0.543
M Relevance Accuracy on test data:  0.568


In [21]:
# Accuracies for Relevance

print("Naive Bayes with Topic and Hashtags: ",versionRelevance1NBacc)
print("Maxent with Topic and Hashtags: ",versionRelevance1Macc)
print("Naive Bayes with Hashtags: ",versionRelevance2NBacc)
print("Maxent with Hashtags: ",versionRelevance2Macc)
print("Naive Bayes with Topic: ",versionRelevance3NBacc)
print("Maxent with Topic: ",versionRelevance3Macc)
print("Naive Bayes without either: ",versionRelevance4NBacc)
print("Maxent without either: ",versionRelevance4Macc)

Naive Bayes with Topic and Hashtags:  0.647
Maxent with Topic and Hashtags:  0.769
Naive Bayes with Hashtags:  0.706
Maxent with Hashtags:  0.781
Naive Bayes with Topic:  0.543
Maxent with Topic:  0.546
Naive Bayes without either:  0.543
Maxent without either:  0.568


In [12]:
import pickle
with open('relevance_model.pkl', 'wb') as file: 
    # Serialize the data and write it to the file 
    pickle.dump(classifierRelevance2M, file) 

# Getting Relevant Tweets

In [22]:
train_relevant_tweets = train_tweets[train_tweets["Relevance"] == "Relevant"]
test_relevant_tweets = test_tweets[test_tweets["Relevance"] == "Relevant"]
validate_relevant_tweets = validate_tweets[validate_tweets["Relevance"] == "Relevant"]

# Naive Bayes and Maxent Classification

With Topic and Hashtags

In [23]:
# Train Data with Topic and Hashtags

train_labels = train_relevant_tweets["Sentiment"].tolist()
train_hashtags = train_relevant_tweets["hashtags"].tolist()
train_hashtags = [str(hashtags).split(",") for hashtags in train_hashtags] 
train_topics = train_relevant_tweets["Topic Label"].tolist()
train_corpus = list(train_relevant_tweets["token"])
train_corpus = [eval(i) for i in train_corpus] 

train_dictionary = gensim.corpora.Dictionary(train_corpus)

labeled_training_data = []
for (l, s, t, h) in zip(train_labels, train_corpus, train_topics, train_hashtags):
    vector = train_dictionary.doc2bow(s)
    sent_as_dict = {id:1 for (id, tf) in vector}
    sent_as_dict[t] = 1
    for tag in h:
        sent_as_dict[tag] = 1
    labeled_training_data.append((sent_as_dict, l))
    
classifier1NB = nltk.NaiveBayesClassifier.train(labeled_training_data)
classifier1M = nltk.MaxentClassifier.train(labeled_training_data)

  ==> Training (100 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -1.09861        0.231
             2          -0.74410        0.848
             3          -0.58586        0.902
             4          -0.49051        0.925
             5          -0.42581        0.940
             6          -0.37840        0.952
             7          -0.34181        0.959
             8          -0.31253        0.966
             9          -0.28845        0.969
            10          -0.26823        0.971
            11          -0.25097        0.973
            12          -0.23603        0.974
            13          -0.22295        0.976
            14          -0.21139        0.976
            15          -0.20110        0.978
            16          -0.19186        0.978
            17          -0.18353        0.979
            18          -0.17596        0.981
            19          -0.16905        0.981
 

In [24]:
# Test Data with Topic and Hashtags

test_labels = test_relevant_tweets["Sentiment"].tolist()
test_hashtags = test_relevant_tweets["hashtags"].tolist()
test_hashtags = [str(hashtags).split(",") for hashtags in test_hashtags] 
test_topics = test_relevant_tweets["Topic Label"].tolist()
test_corpus = list(test_relevant_tweets["token"])
test_corpus = [eval(i) for i in test_corpus] 

test_dictionary = gensim.corpora.Dictionary(test_corpus)

labeled_test_data = []
for (l, s, t, h) in zip(test_labels, test_corpus, test_topics, test_hashtags):
    vector = test_dictionary.doc2bow(s)
    sent_as_dict = {id:1 for (id, tf) in vector}
    sent_as_dict[t] = 1
    for tag in h:
        sent_as_dict[tag] = 1
    labeled_test_data.append((sent_as_dict, l))
    
print("NB Accuracy on test data: ", nltk.classify.accuracy(classifier1NB, labeled_test_data))
version1NBacc = nltk.classify.accuracy(classifier1NB, labeled_test_data)
print("M Accuracy on test data: ", nltk.classify.accuracy(classifier1M, labeled_test_data))
version1Macc = nltk.classify.accuracy(classifier1M, labeled_test_data)

NB Accuracy on test data:  0.3963963963963964
M Accuracy on test data:  0.44015444015444016


With Hashtags only

In [25]:
# Train Data with Hashtags

train_labels = train_relevant_tweets["Sentiment"].tolist()
train_hashtags = train_relevant_tweets["hashtags"].tolist()
train_hashtags = [str(hashtags).split(",") for hashtags in train_hashtags] 
train_corpus = list(train_relevant_tweets["token"])
train_corpus = [eval(i) for i in train_corpus] 

train_dictionary = gensim.corpora.Dictionary(train_corpus)

labeled_training_data = []
for (l, s, h) in zip(train_labels, train_corpus, train_hashtags):
    vector = train_dictionary.doc2bow(s)
    sent_as_dict = {id:1 for (id, tf) in vector}
    for tag in h:
        sent_as_dict[tag] = 1
    labeled_training_data.append((sent_as_dict, l))
    
classifier2NB = nltk.NaiveBayesClassifier.train(labeled_training_data)
classifier2M = nltk.MaxentClassifier.train(labeled_training_data)

  ==> Training (100 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -1.09861        0.231
             2          -0.72689        0.855
             3          -0.56718        0.910
             4          -0.47290        0.929
             5          -0.40952        0.944
             6          -0.36331        0.957
             7          -0.32776        0.965
             8          -0.29938        0.969
             9          -0.27608        0.969
            10          -0.25655        0.973
            11          -0.23989        0.974
            12          -0.22549        0.975
            13          -0.21291        0.976
            14          -0.20180        0.977
            15          -0.19191        0.978
            16          -0.18304        0.980
            17          -0.17504        0.981
            18          -0.16779        0.982
            19          -0.16118        0.982
 

In [26]:
# Test Data with Hashtags

test_labels = test_relevant_tweets["Sentiment"].tolist()
test_hashtags = test_relevant_tweets["hashtags"].tolist()
test_hashtags = [str(hashtags).split(",") for hashtags in test_hashtags] 
test_corpus = list(test_relevant_tweets["token"])
test_corpus = [eval(i) for i in test_corpus] 

test_dictionary = gensim.corpora.Dictionary(test_corpus)

labeled_test_data = []
for (l, s, h) in zip(test_labels, test_corpus, test_hashtags):
    vector = test_dictionary.doc2bow(s)
    sent_as_dict = {id:1 for (id, tf) in vector}
    for tag in h:
        sent_as_dict[tag] = 1
    labeled_test_data.append((sent_as_dict, l))
    
print("NB Accuracy on test data: ", nltk.classify.accuracy(classifier2NB, labeled_test_data))
version2NBacc = nltk.classify.accuracy(classifier2NB, labeled_test_data)
print("M Accuracy on test data: ", nltk.classify.accuracy(classifier2M, labeled_test_data))
version2Macc = nltk.classify.accuracy(classifier2M, labeled_test_data)

NB Accuracy on test data:  0.38481338481338484
M Accuracy on test data:  0.42342342342342343


With Topic Only

In [27]:
# Train Data with Topic

train_labels = train_relevant_tweets["Sentiment"].tolist()
train_topics = train_relevant_tweets["Topic Label"].tolist()
train_corpus = list(train_relevant_tweets["token"])
train_corpus = [eval(i) for i in train_corpus] 

train_dictionary = gensim.corpora.Dictionary(train_corpus)

labeled_training_data = []
for (l, s, t) in zip(train_labels, train_corpus, train_topics):
    vector = train_dictionary.doc2bow(s)
    sent_as_dict = {id:1 for (id, tf) in vector}
    sent_as_dict[t] = 1
    labeled_training_data.append((sent_as_dict, l))
    
classifier3NB = nltk.NaiveBayesClassifier.train(labeled_training_data)
classifier3M = nltk.MaxentClassifier.train(labeled_training_data)

  ==> Training (100 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -1.09861        0.231
             2          -0.76351        0.837
             3          -0.61025        0.896
             4          -0.51639        0.919
             5          -0.45183        0.936
             6          -0.40403        0.948
             7          -0.36685        0.952
             8          -0.33690        0.958
             9          -0.31215        0.961
            10          -0.29128        0.964
            11          -0.27340        0.966
            12          -0.25788        0.966
            13          -0.24426        0.968
            14          -0.23220        0.970
            15          -0.22143        0.970
            16          -0.21176        0.971
            17          -0.20301        0.972
            18          -0.19505        0.974
            19          -0.18778        0.974
 

In [28]:
# Test Data with Topic

test_labels = test_relevant_tweets["Sentiment"].tolist()
test_topics = test_relevant_tweets["Topic Label"].tolist()
test_corpus = list(test_relevant_tweets["token"])
test_corpus = [eval(i) for i in test_corpus] 

test_dictionary = gensim.corpora.Dictionary(test_corpus)

labeled_test_data = []
for (l, s, t) in zip(test_labels, test_corpus, test_topics):
    vector = test_dictionary.doc2bow(s)
    sent_as_dict = {id:1 for (id, tf) in vector}
    sent_as_dict[t] = 1
    labeled_test_data.append((sent_as_dict, l))
    
print("NB Accuracy on test data: ", nltk.classify.accuracy(classifier3NB, labeled_test_data))
version3NBacc = nltk.classify.accuracy(classifier3NB, labeled_test_data)
print("M Accuracy on test data: ", nltk.classify.accuracy(classifier3M, labeled_test_data))
version3Macc = nltk.classify.accuracy(classifier3M, labeled_test_data)

NB Accuracy on test data:  0.31917631917631917
M Accuracy on test data:  0.3410553410553411


Without Either

In [29]:
# Train Data with Topic

train_labels = train_relevant_tweets["Sentiment"].tolist()
train_corpus = list(train_relevant_tweets["token"])
train_corpus = [eval(i) for i in train_corpus] 

train_dictionary = gensim.corpora.Dictionary(train_corpus)

labeled_training_data = []
for (l, s) in zip(train_labels, train_corpus):
    vector = train_dictionary.doc2bow(s)
    sent_as_dict = {id:1 for (id, tf) in vector}
    labeled_training_data.append((sent_as_dict, l))
    
classifier4NB = nltk.NaiveBayesClassifier.train(labeled_training_data)
classifier4M = nltk.MaxentClassifier.train(labeled_training_data)

  ==> Training (100 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -1.09861        0.231
             2          -0.74311        0.853
             3          -0.58752        0.903
             4          -0.49456        0.927
             5          -0.43138        0.939
             6          -0.38492        0.949
             7          -0.34895        0.955
             8          -0.32007        0.960
             9          -0.29627        0.962
            10          -0.27625        0.964
            11          -0.25912        0.966
            12          -0.24429        0.967
            13          -0.23128        0.969
            14          -0.21978        0.971
            15          -0.20953        0.972
            16          -0.20032        0.973
            17          -0.19200        0.974
            18          -0.18445        0.977
            19          -0.17755        0.978
 

In [30]:
# Test Data with Topic

test_labels = test_relevant_tweets["Sentiment"].tolist()
test_corpus = list(test_relevant_tweets["token"])
test_corpus = [eval(i) for i in test_corpus] 

test_dictionary = gensim.corpora.Dictionary(test_corpus)

labeled_test_data = []
for (l, s) in zip(test_labels, test_corpus):
    vector = test_dictionary.doc2bow(s)
    sent_as_dict = {id:1 for (id, tf) in vector}
    labeled_test_data.append((sent_as_dict, l))
    
print("NB Accuracy on test data: ", nltk.classify.accuracy(classifier4NB, labeled_test_data))
version4NBacc = nltk.classify.accuracy(classifier4NB, labeled_test_data)
print("M Accuracy on test data: ", nltk.classify.accuracy(classifier4M, labeled_test_data))
version4Macc = nltk.classify.accuracy(classifier4M, labeled_test_data)

NB Accuracy on test data:  0.323037323037323
M Accuracy on test data:  0.32432432432432434


In [31]:
# Accuracies

print("Naive Bayes with Topic and Hashtags: ",version1NBacc)
print("Maxent with Topic and Hashtags: ",version1Macc)
print("Naive Bayes with Hashtags: ",version2NBacc)
print("Maxent with Hashtags: ",version2Macc)
print("Naive Bayes with Topic: ",version3NBacc)
print("Maxent with Topic: ",version3Macc)
print("Naive Bayes without either: ",version4NBacc)
print("Maxent without either: ",version4Macc)

Naive Bayes with Topic and Hashtags:  0.3963963963963964
Maxent with Topic and Hashtags:  0.44015444015444016
Naive Bayes with Hashtags:  0.38481338481338484
Maxent with Hashtags:  0.42342342342342343
Naive Bayes with Topic:  0.31917631917631917
Maxent with Topic:  0.3410553410553411
Naive Bayes without either:  0.323037323037323
Maxent without either:  0.32432432432432434


# Validation

In [32]:
# Validation Data with Topic and Hashtags

test_labels = validate_relevant_tweets["Sentiment"].tolist()
test_hashtags = validate_relevant_tweets["hashtags"].tolist()
test_hashtags = [str(hashtags).split(",") for hashtags in test_hashtags] 
test_topics = validate_relevant_tweets["Topic Label"].tolist()
test_corpus = list(validate_relevant_tweets["token"])
test_corpus = [eval(i) for i in test_corpus] 

test_dictionary = gensim.corpora.Dictionary(test_corpus)

labeled_test_data = []
for (l, s, t, h) in zip(test_labels, test_corpus, test_topics, test_hashtags):
    vector = test_dictionary.doc2bow(s)
    sent_as_dict = {id:1 for (id, tf) in vector}
    sent_as_dict[t] = 1
    for tag in h:
        sent_as_dict[tag] = 1
    labeled_test_data.append((sent_as_dict, l))

print("NB Accuracy on validation data: ", nltk.classify.accuracy(classifier1NB, labeled_test_data))
version1NBacc = nltk.classify.accuracy(classifier1NB, labeled_test_data)
print("M Accuracy on validation data: ", nltk.classify.accuracy(classifier1M, labeled_test_data))
version1Macc = nltk.classify.accuracy(classifier1M, labeled_test_data)

NB Accuracy on validation data:  0.4426229508196721
M Accuracy on validation data:  0.49453551912568305


In [33]:
import pickle
with open('sentiment_model.pkl', 'wb') as file: 
    # Serialize the data and write it to the file 
    pickle.dump(classifier1M, file) 

# EmoRoberta

In [2]:
from transformers import pipeline
import tensorflow

In [3]:
# download pre-trained emotion classification model
emorobertamodel = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base")

In [13]:
tweet_text = labelled_tweets["text"].tolist()
emotions = emorobertamodel(tweet_text)

ValueError: Length of values (5000) does not match length of index (494540)

In [16]:
tweets["emotion_label"] = ""
tweets["emotion_score"] = ""

In [17]:
emotions
tweets["emotion_label"].iloc[0:5000] = [d["label"] for d in emotions]
tweets["emotion_score"].iloc[0:5000] = [d["score"] for d in emotions]

C:\Users\99zhe\AppData\Local\Temp\ipykernel_44372\692127230.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets["emotion_label"].iloc[0:5000] = [d["label"] for d in emotions]
C:\Users\99zhe\AppData\Local\Temp\ipykernel_44372\692127230.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets["emotion_score"].iloc[0:5000] = [d["score"] for d in emotions]


In [18]:
tweets

,userid,tweetid,text,hashtags,token,processed_text,Topic Label,Topic Weight,Sentiment,Relevance,emotion_label,emotion_score
0,5.467372e+07,1.627620e+18,repdavid i support 100,"SlavaUkraïni,JoeBiden","['repdavid', 'support']",repdavid support,International Support and Public Opinion,0.389973,Positive,Relevant,joy,0.520785
1,2.282984e+09,1.611400e+18,province editorial inequality hounds s populat...,"Canada,disabled,Vancouver,homeless","['province', 'editorial', 'inequality', 'hound...",province editorial inequality hound population...,Strategic Military Aspects,0.464470,Irrelevant,Irrelevant,anger,0.661632
2,2.830324e+08,1.636650e+18,they went to war when russia invaded ukraine t...,ArmUkraineNow,"['went', 'war', 'russia', 'invaded', 'ukraine'...",went war russia invaded ukraine believed freed...,Strategic Military Aspects,0.613352,Positive,Relevant,anger,0.605659
3,1.257760e+18,1.626920e+18,on mintwave radio mint2dry,"nowplaying,alexa,streema,Scotland,echo,global,...","['mintwave', 'radio', 'mintdry']",mintwave radio mintdry,Military Actions and Operations,0.279435,Irrelevant,Irrelevant,neutral,0.862803
4,9.241172e+08,1.640020e+18,military historian douglas macgregor why will ...,"NATO,Russia,Kiev","['military', 'historian', 'douglas', 'macgrego...",military historian douglas macgregor crumble d...,Broad Overview of the Conflict,0.434376,Negative,Relevant,fear,0.826465
...,...,...,...,...,...,...,...,...,...,...,...,...
494535,1.327670e+18,1.638460e+18,as a result of this nights attack on kyiv regi...,"RussiaIsATerroristState,ArmUkraineNow","['result', 'night', 'attack', 'kyiv', 'region'...",result night attack kyiv region russian shelle...,Military Actions and Operations,0.485424,NaN,Irrelevant,,
494536,3.915855e+08,1.659220e+18,computools has been recognized as one of the t...,"development,mobileappdevelopment,canada,services","['computools', 'recognized', 'mobile', 'app', ...",computools recognized mobile app developer can...,International Support and Public Opinion,0.394642,NaN,Irrelevant,,
494537,1.001233e+08,1.635110e+18,good morning remember to wave 👋 when new traff...,"NAFOfellas,SlavaUkraini,СлаваУкраїні,RussiaisA...","['good', 'morning', 'remember', 'wave', 'new',...",good morning remember wave new traffic partici...,International Support and Public Opinion,0.483856,NaN,Irrelevant,,
494538,1.534910e+18,1.619070e+18,colonial trope forces german apology germany h...,"germany,Ukraine,AfricanStream,africa","['colonial', 'trope', 'force', 'german', 'apol...",colonial trope force german apology germany ap...,International Support and Public Opinion,0.309035,NaN,Irrelevant,,


In [20]:
tweets.to_csv("emotion_tweets.csv")